In [1]:
import pandas as pd
df = pd.read_csv('data/devices_sample_qns.csv')
df
meta=True
df

,question,query
0,What is the number of POS terminals in Spain a...,select year_2022 from devices where title like...
1,Which country has the highest number of cards ...,"select reporting_country , sum(coalesce(year_2..."
2,What is the number of cards with a delayed deb...,select sum(year_2022) from devices where func...
3,How many cards in Belgium are able to initiate...,select sum(year_2022) from devices where devic...
4,What is the number of POS terminals in Indonesia?,select sum(year_2022) from devices where title...
5,What is the number of cards with a debit funct...,select sum(year_2022) from devices where funct...
6,How many contactless cards are there in India?,select sum(year_2022) from devices where techn...
7,What is the number of contactless cards in Eng...,select sum(year_2022) from devices where techn...
8,What is the number of cards with an e-money fu...,select sum(year_2022) from devices where funct...
9,Which country has the highest number of cards ...,"select sum(coalesce(year_2022,0)) as total, re..."


In [2]:
texts=df.question.tolist()
metadata=[{'query':query } for query in df['query'].values]


metadata



[{'query': "select year_2022 from devices where title like '%Number of POS terminals%' and reporting_country='Spain' and collection_indicator='End of period';"},
 {'query': "select reporting_country , sum(coalesce(year_2022,0)) as total from devices where function_verbose='Cards with cash function' group by reporting_country order by total desc limit 1"},
 {'query': "select sum(year_2022)\xa0 from devices where function_verbose='Cards with delayed debit function' and reporting_country='Canada'"},
 {'query': "select sum(year_2022) from devices where device_present_device_not_present='Able to initiate device-not-present payments' and reporting_country='Belgium'"},
 {'query': "select sum(year_2022) from devices where title like '%Number of POS terminals%' and reporting_country='Indonesia'"},
 {'query': "select sum(year_2022) from devices where function_verbose='Cards with debit function' and reporting_country='Türkiye'"},
 {'query': "select sum(year_2022) from devices where technology_ver

In [3]:

# do not change chromadb name
ids=[str(i) for i in df.index]
local_directory='./chromadb'


In [4]:
import shutil
try: 
    shutil.rmtree(local_directory)
    print(f"ChromaDB '{local_directory}' deleted successfully.")

except FileNotFoundError: 
    print(f"ChromaDB '{local_directory}' not found.")

ChromaDB './chromadb' not found.


In [ ]:
#change collection name as you want and give it as command line argument(-col) to main.py

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
ef = embedding_functions.SentenceTransformerEmbeddingFunction()

chroma = chromadb.PersistentClient(path=local_directory)
collection_name="devices"
collection = chroma.get_or_create_collection(name=collection_name, embedding_function=ef)
collection.add(documents=texts, metadatas=metadata, ids=ids)

print(f'docs in {collection_name} : ', len(collection.get()['documents']))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


docs in devices :  10


In [7]:
query_result = collection.query(query_texts=['What is the number of POS terminals in Belgium?'], n_results=2)

In [8]:
query_result

{'ids': [['4', '0']],
 'embeddings': None,
 'documents': [['What is the number of POS terminals in Indonesia?',
   'What is the number of POS terminals in Spain as of the end of the reporting period?']],
 'uris': None,
 'data': None,
 'metadatas': [[{'query': "select sum(year_2022) from devices where title like '%Number of POS terminals%' and reporting_country='Indonesia'"},
   {'query': "select year_2022 from devices where title like '%Number of POS terminals%' and reporting_country='Spain' and collection_indicator='End of period';"}]],
 'distances': [[0.32367157294965254, 0.45506188966841393]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [9]:
few_shots='Examples \n'
for i,j in zip(query_result['documents'][0],query_result['metadatas'][0]):
    few_shots=few_shots+'queston : '+i +'\n'+'query : '+j['query']+'\n'
print(few_shots)

Examples 
queston : What is the number of POS terminals in Indonesia?
query : select sum(year_2022) from devices where title like '%Number of POS terminals%' and reporting_country='Indonesia'
queston : What is the number of POS terminals in Spain as of the end of the reporting period?
query : select year_2022 from devices where title like '%Number of POS terminals%' and reporting_country='Spain' and collection_indicator='End of period';



In [10]:
if not False:
    print(7)

7
